# Prezentacja 3

In [1]:
from pandas import read_csv, to_datetime, read_excel, Series, date_range, DataFrame
import requests
import datetime
import csv
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as stats
from scipy.stats import t, norm
from scipy.integrate import quad
from scipy.optimize import curve_fit
from statsmodels.stats.diagnostic import lilliefors

### Średnia cena ziemniaka (2010-2024)

https://www.dlahandlu.pl/koszyk/towar/ziemniaki-na-wage-1-kg,35.html

In [2]:
dane_ziemniak = [
    1.37,1.50,1.66,1.81,1.95,2.15,1.70,1.44,1.45,1.43,1.52,1.67, #2010
    1.82,2.14,2.27,2.46,2.47,2.07,1.33,0.85,0.83,0.77,0.80,0.78, #2011
    0.78,0.76,0.89,0.83,0.97,1.20,0.89,0.70,0.72,0.70,0.81,0.97, #2012
    1.07,1.21,1.27,1.35,1.83,2.00,1.54,1.33,1.34,1.41,1.39,1.47, #2013
    1.55,1.72,1.71,1.68,1.86,1.74,0.90,0.93,0.76,0.88,0.91,0.75, #2014
    0.87,0.89,0.82,0.89,1.28,1.41,1.05,1.03,1.08,1.16,1.15,1.28, #2015
    1.37,1.47,1.55,1.82,2.24,2.09,1.34,1.17,1.00,1.11,1.39,1.20, #2016
    1.34,1.13,1.28,1.67,2.09,2.21,1.34,1.38,1.32,1.45,1.24,1.36, #2017
    1.54,1.55,1.49,1.61,2.20,1.41,1.69,1.38,1.67,1.53,1.97,1.83, #2018
    2.07,2.15,2.48,2.82,3.82,3.74,3.28,2.76,3.03,2.58,2.90,2.36, #2019
    3.03,3.03,3.23,2.92,2.60,2.27,2.29,1.40,1.43,1.52,1.39,1.41, #2020
    1.56,1.66,1.91,2.41,1.65,3.06,2.12,1.77,2.11,2.15,2.04,2.26, #2021
    2.41,2.19,2.17,2.00,2.32,2.39,2.38,2.21,1.84,2.14,2.72,2.17, #2022
    2.14,2.10,2.31,2.21,2.51,3.36,3.67,2.64,2.91,2.55,2.56,2.48, #2023
    2.78,2.43,3.09,3.46,3.26,3.50,3.29,2.83,2.86,2.45,2.76,2.68 #2024
]
daty_ziemniak = date_range(start="2010-01", end="2024-12", freq="MS")
# ze względu na brak danych w kwietniu 2020 uśredniono dane z marca i maja 2020

### Otyłość WHO (procent w Polsce, lata 1990-2022)

https://data.who.int/indicators/i/C6262EC/BEFA58B

In [4]:
dane_otylosc_global = read_csv('dane\\otylosc_who.csv')
dane_otylosc = dane_otylosc_global[
    (dane_otylosc_global["GEO_NAME_SHORT"] == "Poland") &
    (dane_otylosc_global["DIM_SEX"] == "TOTAL")
]
dane_otylosc = dane_otylosc.sort_values(by="DIM_TIME")
dane_otylosc = dane_otylosc["RATE_PER_100_N"].tolist()
daty_otylosc = list(range(1990, 2023))

### Cena cukru (dziennie, 2010-2024)

https://www.dlahandlu.pl/koszyk/towar/cukier-bialy-krysztal-1-kg,22.html

In [5]:
dane_cukier = [
    2.55,2.52,2.45,2.45,2.41,2.38,2.33,2.69,2.89,2.86,2.77,2.86, # 2010
    3.01,3.43,4.71,4.72,4.33,3.93,3.93,3.61,3.68,3.61,3.72,3.61, # 2011
    3.78,3.79,3.79,3.84,3.80,3.84,3.80,3.77,3.78,3.73,3.78,3.76, # 2012
    3.67,3.53,3.67,3.55,3.55,3.47,3.33,3.22,3.14,3.15,3.01,2.85, # 2013
    2.83,2.73,2.58,2.35,2.18,2.24,2.04,2.16,1.93,1.98,1.89,1.81, # 2014
    1.85,1.85,1.98,2.00,1.98,1.94,2.20,2.21,2.30,2.28,2.33,2.34, # 2015
    2.32,2.55,2.60,2.61,2.67,2.65,2.91,2.82,3.07,2.96,3.11,2.98, # 2016
    3.16,2.95,2.93,3.14,2.97,3.12,2.97,3.06,2.84,2.71,2.87,2.46, # 2017
    2.02,2.48,1.96,2.41,1.96,1.91,2.14,2.00,2.03,2.21,2.06,1.87, # 2018
    2.36,2.36,2.57,2.34,2.65,2.37,2.62,2.47,2.68,2.42,2.73,2.42, # 2019
    2.70,2.50,2.54,2.63,2.72,2.66,2.47,2.65,2.38,2.38,2.59,2.57, # 2020
    2.75,2.42,2.53,2.81,2.61,2.51,3.01,2.44,2.53,3.11,3.03,3.33, # 2021
    3.06,3.06,3.33,3.34,3.93,3.33,4.25,5.49,6.76,5.65,6.64,6.00, # 2022
    6.68,6.02,6.63,6.02,6.59,5.90,6.54,5.81,6.14,5.60,6.03,5.70, # 2023
    6.12,4.76,4.90,5.09,4.55,5.31,4.40,4.86,3.65,4.53,3.51,3.30] # 2024
daty_cukier = date_range(start="2010-01", end="2024-12", freq="MS")
# ze względu na brak danych w kwietniu 2020 uśredniono dane z marca i maja 2020

###  Notowania giełdowe PEPSICO (PEP) - od 13.04.2015 do 31.03.2025, co tydzień

https://www.nasdaq.com/market-activity/stocks/pep/historical?page=1&rows_per_page=10&timeline=y10

In [6]:
dane_notowania = read_csv('dane\\notowania.csv')
dane_notowania['Date'] = to_datetime(dane_notowania['Date'], format='%m/%d/%Y')
dane_notowania = dane_notowania.sort_values('Date')
dane_notowania.set_index('Date', inplace=True)
# Ustalenie startowej daty: 13.04.2015
start_date = to_datetime('2015-04-13')
dane_notowania = dane_notowania[dane_notowania.index >= start_date]
dane_notowania = dane_notowania.resample('7D').first()
daty_notowania = dane_notowania.index.tolist()
dane_notowania = dane_notowania["Close/Last"].tolist()
dane_notowania = [float(x.replace('$', '')) for x in dane_notowania]
# aby uniknąć problemu związanemu z brakiem danych w weekendy i dni niehandlowe, bierzemy notowanie z każdego poniedziałku (jeśli nie ma z danego poniedziałku, to z wtorku etc.)

### Kursy walut (07.2020-04.2025)

1) Bierzemy pod uwagę kurs sprzedaży obcej waluty (np. euro na pln) - zakładamy, że faktura wystawiona jest w obcej walucie
2) Złym scenariuszem dla nas będzie, jeśli wartość portfela spada
4) Bierzemy pod uwagę eksport PepsiCo Polska do Niemiec, Czech, Węgier i Słowacji (50% euro, 25% korona czeska, 25% forint węgierski)
5) Ponownie bierzemy kurs z każdego poniedziałku

https://www.bankier.pl/narzedzia/archiwum-kursow-walutowych

### Euro

In [8]:
dane = read_csv('dane\\euro.csv', sep=';', encoding='windows-1250', skiprows=1)
# Usuń wiersze, które nie mają poprawnej daty
dane = dane[to_datetime(dane['Data'], errors='coerce').notna()]
dane['Data'] = to_datetime(dane['Data'], format='%Y-%m-%d')
dane = dane.sort_values('Data')
dane.set_index('Data', inplace=True)
start_date = to_datetime('2020-07-13')
dane = dane[dane.index >= start_date]
dane = dane.resample('7D').first()
daty_waluty = dane.index.tolist()
dane_euro = dane["Kurs"].tolist()

### CZK

In [10]:
dane = read_csv('dane\\czk.csv', sep=';', encoding='windows-1250', skiprows=1)
# Usuń wiersze, które nie mają poprawnej daty
dane = dane[to_datetime(dane['Data'], errors='coerce').notna()]
dane['Data'] = to_datetime(dane['Data'], format='%Y-%m-%d')
dane = dane.sort_values('Data')
dane.set_index('Data', inplace=True)
start_date = to_datetime('2020-07-13')
dane = dane[dane.index >= start_date]
dane = dane.resample('7D').first()
dane_czk = dane["Kurs"].tolist()

### HUF

In [11]:
dane = read_csv('dane\\huf.csv', sep=';', encoding='windows-1250', skiprows=1)
# Usuń wiersze, które nie mają poprawnej daty
dane = dane[to_datetime(dane['Data'], errors='coerce').notna()]
dane['Data'] = to_datetime(dane['Data'], format='%Y-%m-%d')
dane = dane.sort_values('Data')
dane.set_index('Data', inplace=True)
start_date = to_datetime('2020-07-13')
dane = dane[dane.index >= start_date]
dane = dane.resample('7D').first()
dane_huf = dane["Kurs"].tolist()

### Portfel

In [12]:
wagi = np.array([0.5, 0.25, 0.25])
dane = np.array([dane_euro, dane_czk, dane_huf])
n = len(dane)
dane_waluty = sum([wagi[i]*dane[i] for i in range(n)])
dane_waluty = dane_waluty.tolist()

### Zysk netto Pepsico Inc. (12.2009-12.2024)

https://www.macrotrends.net/stocks/charts/PEP/pepsico/gross-profit

Dane w milionach dolarów

In [16]:
dane = read_excel("dane/zysk_netto.xlsx", header=None, names=["Data", "Cena"])
dane["Data"] = to_datetime(dane["Data"], format="%d.%m.%Y")
# Usuń symbol dolara i przecinki, zamień na float
dane["Cena"] = dane["Cena"].replace(r'[\$,]', '', regex=True).astype(float)
df_sorted = dane.sort_values("Data")
daty_zysk = df_sorted["Data"].tolist()
dane_zysk = df_sorted["Cena"].tolist()

In [17]:
print("1) Dane firmy matki:")
print("1.1) Notowania giełdowe: ")
print(dane_notowania)
print("1.2) Zysk brutto: ")
print(dane_zysk)
print("2) Dane walutowe: ")
print(dane_waluty)
print("3) Dane produktów: ")
print("3.1) Ceny ziemniaka: ")
print(dane_ziemniak)
print("3.2) Cena cukru: ")
print(dane_cukier)
print("4) Dane otyłość: ")
print(dane_otylosc)

1) Dane firmy matki:
1.1) Notowania giełdowe: 
[95.95, 96.35, 98.05, 95.79, 95.97, 92.84, 92.64, 95.6, 93.69, 94.4, 97.22, 97.31, 95.66, 97.45, 99.09, 99.86, 91.83, 92.93, 92.37, 91.2, 93.71, 92.47, 95.81, 99.14, 100.18, 102.54, 101.4, 98.88, 99.56, 100.85, 100.16, 100.67, 97.95, 98.62, 100.68, 98.77, 97.44, 95.77, 95.94, 99.03, 97.52, 98.6, 99.87, 97.82, 99.25, 100.65, 101.54, 100.98, 103.82, 104.3, 103.72, 102.8, 103.43, 105.28, 104.13, 100.42, 101.17, 102.7, 103.06, 103.37, 102.13, 106.42, 108.91, 109.21, 109.02, 108.93, 108.14, 108.72, 107.96, 107.99, 107.92, 106.02, 105.67, 107.03, 108.25, 106.22, 106.85, 107.31, 107.2, 106.96, 101.39, 102.26, 102.79, 100.71, 104.72, 105.41, 105.13, 104.65, 103.46, 102.19, 104.01, 103.69, 104.57, 106.52, 109.36, 109.27, 109.63, 109.41, 111.15, 111.83, 111.86, 112.02, 113.32, 114.32, 112.28, 113.22, 113.94, 114.7, 118.01, 117.88, 116.23, 117.29, 117.12, 115.44, 114.27, 114.93, 116.13, 116.61, 116.96, 117.84, 118.26, 115.54, 115.87, 115.69, 114.99, 

## Metody wyznaczania VaR

### Metoda parametryczna

In [30]:
# Obliczanie tygodniowych zmian procentowych
zmiany_procentowe = Series(dane_notowania).pct_change().dropna() * 100
daty_zmian = daty_notowania[1:]  # pierwszy tydzień nie ma zmiany

# Unormowane dane lub oryginalne zmiany procentowe:
data = np.array(zmiany_procentowe)

# Dopasuj rozkład t: zwraca df, loc, scale
df_fit, loc_fit, scale_fit = t.fit(data)
# print(f"Dopasowane parametry rozkładu t:\n - df: {df_fit:.2f}\n - loc: {loc_fit:.4f}\n - scale: {scale_fit:.4f}")

print("VaR rzędu 95% dla notowań:", -t.ppf(0.05, df_fit, loc=loc_fit, scale=scale_fit))
print("VaR rzędu 99% dla notowań:", -t.ppf(0.01, df_fit, loc=loc_fit, scale=scale_fit))

VaR rzędu 95% dla notowań: 3.507347877298922
VaR rzędu 99% dla notowań: 5.875055254739729


### Metoda historyczna (zwykła)

In [29]:
print("Kwantyl empiryczny dla danych:", -np.quantile(data, 0.05))
print("Kwantyl empiryczny dla danych:", -np.quantile(data, 0.01))

Kwantyl empiryczny dla danych: 3.5473920358945366
Kwantyl empiryczny dla danych: 5.6245881905998685


In [42]:
def weighted_historical_VaR(data, alpha, lambd=0.96):
    n = len(data)
    w_1 = (1 - lambd) / (1 - np.power(lambd, n))
    weights = np.power(lambd, n - np.arange(1, n + 1))  * w_1
    return -np.quantile(data, alpha, weights=weights, method="inverted_cdf")

print(weighted_historical_VaR(data, 0.05))
print(weighted_historical_VaR(data, 0.01))

4.501800720288118
5.623368376490512
